<a href="https://colab.research.google.com/github/kyungminkim-dev/boostcamp-ai-tech/blob/main/10_transformers_2_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**10. Transformers 라이브러리 사용하기 2**
1. GPT-2 모델 및 tokenizer 실습.
2. Special token 추가.

### **필요 패키지 import**

In [ ]:
!pip install transformers

In [ ]:
from transformers import *
from torch import nn
from tqdm import tqdm

import torch

### **GPT-2 불러오기**

In [ ]:
gpt_name = 'gpt2'

In [ ]:
config = GPT2Config.from_pretrained(gpt_name)
tokenizer = GPT2Tokenizer.from_pretrained(gpt_name)
model = GPT2Model.from_pretrained(gpt_name)

In [ ]:
config

In [ ]:
tokenizer

In [ ]:
model

### **Tokenizer 사용**

In [ ]:
sentence = "I want to go home."
output = tokenizer(sentence)

In [ ]:
output

In [ ]:
tokenized = tokenizer.tokenize(sentence)

In [ ]:
tokenized

In [ ]:
vocab = tokenizer.get_vocab()

In [ ]:
print(vocab)
print(len(vocab))

In [ ]:
vocab['<|endoftext|>']

In [ ]:
token_ids = [vocab[token] for token in tokenized]
print(token_ids)

In [ ]:
token_ids = [tokenizer._convert_token_to_id(token) for token in tokenized]
print(token_ids)

In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokenized)
print(token_ids)

In [ ]:
token_ids = tokenizer.encode(sentence)
print(token_ids)

In [ ]:
sentence = tokenizer.convert_tokens_to_string(tokenized)
print(sentence)

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(token_ids)
print(tokens)
sentence = tokenizer.convert_tokens_to_string(tokens)
print(sentence)

### **데이터 전처리**

마찬가지로 sample 데이터를 전처리합니다.

In [ ]:
data = [
  "I want to go home.",
  "My dog's name is Max.",
  "Natural Language Processing is my favorite research field.",
  "Welcome. How can I help you?",
  "Shoot for the moon. Even if you miss, you'll land among the stars."
]

In [ ]:
max_len = 0
batch = []

for sent in tqdm(data):
  token_ids = tokenizer.encode(sent)
  max_len = max(max_len, len(token_ids))
  batch.append(token_ids)

In [ ]:
pad_id = 0

for i, token_ids in enumerate(tqdm(batch)):
  if len(token_ids) < max_len:
    batch[i] = token_ids + [pad_id] * (max_len-len(token_ids))

In [ ]:
batch = torch.LongTensor(batch)

print(batch)
print(batch.shape)

In [ ]:
batch_mask = (batch != pad_id).float()

print(batch_mask)
print(batch.shape)

### **GPT-2 사용 및 응용**

In [ ]:
outputs = model(input_ids=batch, attention_mask=batch_mask)

In [ ]:
outputs

In [ ]:
# B: batch size, L: max length, d_h: hidden size
last_hidden_states = outputs[0]  # (B, L, d_h)

print(last_hidden_states.shape)

다음과 같이 fully connected layer를 하나 사용하여 다음 단어를 예측할 수 있습니다.

In [ ]:
lm_linear = nn.Linear(config.hidden_size, config.vocab_size)

In [ ]:
# V: vocab size
lm_output = lm_linear(last_hidden_states)  # (B, L, V)

print(lm_output)
print(lm_output.shape)

GPT-2 역시 다양한 head와 제공됩니다. (https://huggingface.co/transformers/model_doc/gpt2.html)  
위의 Language Modeling을 동일하게 수행할 수 있는 모델은 다음과 같습니다.

In [ ]:
lm_model = GPT2LMHeadModel.from_pretrained(gpt_name)

In [ ]:
lm_model

GPT2LMHeadModel은 `input_ids`와 `labels`를 함께 줄 경우 자동으로 cross entropy loss까지 계산해줍니다.  
`labels`가 주어지지 않을 경우엔 기존과 동일하게 결과만 주어집니다.

In [ ]:
outputs = lm_model(input_ids=batch, attention_mask=batch_mask, labels=batch)

In [ ]:
outputs

In [ ]:
loss = outputs[0]

print(loss)

In [ ]:
logits = outputs[1]

print(logits)
print(logits.shape)

### **Special token 추가하기**

경우에 따라선 별도의 special token을 추가하고 싶을 수 있습니다.  

In [ ]:
print(vocab)
print(len(vocab))

In [ ]:
special_tokens = {
    'bos_token': '[BOS]',
    'eos_token': '[EOS]',
    'pad_token': '[PAD]',
    'additional_special_tokens': ['[SP1]', '[SP2]']
}

In [ ]:
num_new_tokens = tokenizer.add_special_tokens(special_tokens)
print(num_new_tokens)

In [ ]:
vocab = tokenizer.get_vocab()
print(vocab)
print(len(vocab))

In [ ]:
tokenizer

Special token을 추가했다면 거기에 맞게 모델의 embedding layer의 input size도 바꿔주어야 합니다.

In [ ]:
model.resize_token_embeddings(len(vocab))

In [ ]:
model